In [ ]:
%pylab inline

In [ ]:
from __future__ import division
from IPython.display import Image
from deltasigma import *
import warnings
warnings.filterwarnings('ignore')

In [ ]:
np.set_printoptions(suppress=True, precision=3)

In [ ]:
order = 8
osr = 32
nlev = 2
f0 = 0.125
Hinf = 1.5
form = 'CRFB'

In [ ]:
ntf = synthesizeNTF(order, osr, 2, Hinf, f0)            # Optimized zero placement
print("Synthesized a %d-order NTF, with roots:\n" % order)
print(" Zeros:\t\t\t Poles:")
for z, p in zip(ntf[0], ntf[1]):
    print("(%f, %fj)\t(%f, %fj)" % (np.real(z), np.imag(z), np.real(p), np.imag(p)))
print("")

In [ ]:
plotPZ(ntf, showlist=True)
a, g, b, c = realizeNTF(ntf, form)

In [ ]:
Image(url='http://python-deltasigma.readthedocs.org/en/latest/_images/CRFB.png', retina=True)

In [ ]:
b = np.hstack(( # Use a single feed-in for the input
               np.atleast_1d(b[0]),
               np.zeros((b.shape[0] - 1, ))
             ))
ABCD = stuffABCD(a, g, b, c, form)
print("ABCD Matrix:")
print(ABCD)

In [ ]:
DocumentNTF(ABCD, osr, f0)
f = gcf()
f.set_size_inches((15, 6))

In [ ]:
figure(figsize=(15,8))
PlotExampleSpectrum(ntf, M=1, osr=osr, f0=f0)

In [ ]:
snr, amp = simulateSNR(ntf, osr, None, f0, nlev)

In [ ]:
figure(figsize=(15,8))
if nlev == 2 and f0 == 0.:
    snr_pred, amp_pred, k0, k1, se = predictSNR(ntf, osr)
    plot(amp_pred, snr_pred, '-', label='predicted')
    hold(True)
plot(amp, snr,'o-.g', label='simulated')
xlabel('Input Level (dBFS)')
ylabel('SQNR (dB)')
peak_snr, peak_amp = peakSNR(snr, amp)
msg = 'peak SQNR = %4.1fdB  \n@ amp = %4.1fdB  ' % (peak_snr, peak_amp)
text(peak_amp-10,peak_snr,msg, horizontalalignment='right', verticalalignment='center');
msg = 'OSR = %d ' % osr
text(-2, 5, msg, horizontalalignment='right');
hold(False)
figureMagic([-100, 0], 10, None, [0, 80], 10, None, [12, 6], 'Time-Domain Simulations')
legend(loc=2);

In [ ]:
 # Dynamic range scaling
print('Doing dynamic range scaling... ',)
ABCD0 = ABCD.copy()
ABCD, umax, S = scaleABCD(ABCD0, nlev, f0)
#a, g, b, c = mapABCD(ABCD,form);
print('Done.')
print("Maximum input magnitude: %.3f" % umax)

In [ ]:
print('Verifying dynamic range scaling... ',)
u = np.linspace(0, 0.95*umax, 30)
N = 1e4
N0 = 50
test_tone = np.cos(2*np.pi*f0*np.arange(N))
test_tone[:N0] = test_tone[:N0]*(0.5 - 0.5*np.cos(2*np.pi/N0*np.arange(N0)))
maxima = np.zeros((order, u.shape[0]))
for i in np.arange(u.shape[0]):
    ui = u[i]
    v, xn, xmax, y = simulateDSM(ui*test_tone, ABCD, nlev)
    maxima[:, i] = xmax[:, 0]
    if (xmax > 1e2).any(): 
        print('Warning, umax from scaleABCD was too high.')
        umax = ui
        u = u[:i]
        maxima = maxima[:, :i]
        break
print('Done.')
print("Maximum DC input level: %.3f" % umax)

In [ ]:
colors = get_cmap('jet')(np.linspace(0, 1.0, order))
hold(True)
for i in range(order):
    plot(u, maxima[i,:], 'o-', color=colors[i], label='State %d' % (i+1))
grid(True)
figureMagic([0, umax], None, None, [0, 1] , 0.1, 2, [12, 6], 'State Maxima')
xlabel('DC input')
ylabel('Maxima')
legend(loc='best');

In [ ]:
a, g, b, c = mapABCD(ABCD, form)

In [ ]:
adc = {
       'order':order,
       'osr':osr,
       'nlev':nlev,
       'f0':f0,
       'ntf':ntf,
       'ABCD':ABCD,
       'umax':umax,
       'peak_snr':peak_snr,
       'form':form,
       'coefficients':{
                       'a':a,
                       'g':g,
                       'b':b,
                       'c':c
                      }
      }

In [ ]:
print("Final ADC coefficients:")
print("  %s\n   %s" % ('a', adc['coefficients']['a']))
print("  %s\n   %s" % ('g', adc['coefficients']['g']))
print("  %s\n   %s" % ('b', adc['coefficients']['b']))
print("  %s\n   %s" % ('c', adc['coefficients']['c']))

### System version information

In [ ]:
#%install_ext http://raw.github.com/jrjohansson/version_information/master/version_information.py
%load_ext version_information
%reload_ext version_information

%version_information numpy, scipy, matplotlib, deltasigma